In [11]:
import time
import random
from gurobipy import Model, GRB, quicksum
import numpy as np
import pandas as pd
import os
import math

In [12]:
cur_dir = os.getcwd()
parent_dir = os.path.dirname(cur_dir)
print(f'The current directory is: {cur_dir}, the parent directory is: {parent_dir}')
data_dir = os.path.join(parent_dir, 'PVData')
print(f'The data directory is: {data_dir}')

# creat a new folder to save the results
# first, create a new folder named 'result_data' in the parent directory
result_dir = os.path.join(parent_dir, 'result_data')
if not os.path.exists(result_dir):
    os.makedirs(result_dir)
    print(f'The result directory is created: {result_dir}')

fix_panel_dir = os.path.join(result_dir, 'fix_panel')
if not os.path.exists(fix_panel_dir):
    os.makedirs(fix_panel_dir)
    print(f'The fix panel result directory is created: {fix_panel_dir}')

The current directory is: /Users/dtjgp/Projects/GreenAI/SolarPanelData/Code, the parent directory is: /Users/dtjgp/Projects/GreenAI/SolarPanelData
The data directory is: /Users/dtjgp/Projects/GreenAI/SolarPanelData/PVData


In [13]:
# read the data from the csv file
data = pd.read_csv('power_level_time1000.csv')
# get the power_level data
power_level = data['power_level']
# get the time requirment data
time_requirement = data['total_time']

# change the power_level to a list
power_level = power_level.tolist()
print(power_level)
# change the time_requirement to a list
time_requirement = time_requirement.tolist()
print(time_requirement)

# for each time value in the time_requirement list, round the value to the 2nd decimal place
time_requirement = [round(i, 2) for i in time_requirement]
print(time_requirement)

time_round = [math.ceil(i) for i in time_requirement]
print(time_round)
# get the minimum time value in the time_requirement list
min_time = min(time_round)
# get the maximum time value in the time_requirement list
max_time = max(time_round)

print(f'The minimum time value is: {min_time}, the maximum time value is: {max_time}')

[100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320]
[27.53389759858449, 23.618477900822956, 20.858999411265057, 18.54716673162248, 16.8587359852261, 15.65226145585378, 14.77152551545037, 14.389471703105505, 14.207614541053772, 14.080188870429993, 13.996444715393912, 13.918626162740916, 13.903724935319689, 13.817768573760986, 13.77594202094608, 13.758791433440315, 13.746575474739071, 13.74918410513136, 13.74544240368737, 13.664010882377625, 13.690003196398417, 13.690661735004849, 13.897534939977858]
[27.53, 23.62, 20.86, 18.55, 16.86, 15.65, 14.77, 14.39, 14.21, 14.08, 14.0, 13.92, 13.9, 13.82, 13.78, 13.76, 13.75, 13.75, 13.75, 13.66, 13.69, 13.69, 13.9]
[28, 24, 21, 19, 17, 16, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
The minimum time value is: 14, the maximum time value is: 28


In [14]:
# def generate_instance():
#     operations_info = {}
#     # Each tuple in the list corresponds to (num_operations, power_per_operation)
#     processes_details = [(32, 100), (28, 125), (26, 150), (25, 175), (24, 200), (23, 225), (22, 250)]
    
#     for i, (num_operations, power) in enumerate(processes_details):
#         # Set every operation's duration to 1 and power to the specified power
#         operations = [{'power': power, 'duration': 1} for _ in range(num_operations)]
#         # The maximum time for completion is set to the number of operations
#         max_time = num_operations
#         operations_info[i] = {'operations': operations, 'max_time': max_time}
    
#     return operations_info

In [15]:
def generate_instance(power_level, time_requirement):
    operations_info = {}

    for i in range(len(power_level)):
        # Set every operation's duration to 1 and power to the specified power
        time_count = math.ceil(time_requirement[i])
        operations = [{'power': power_level[i], 'duration': 1} for _ in range(time_count)]
        # The maximum time for completion is set to the number of operations
        max_time = time_requirement[i]
        operations_info[i] = {'operations': operations, 'max_time': max_time}   
    
    return operations_info

In [16]:
def solve_gurobi(
    start_hour, 
    end_hour, 
    instance,
    contributions,
    E_max, alpha, beta, theta, 
    G_max_solar, 
    G_max_grid, 
    GPU_power_options, 
    electricity_price, 
    sell_back_price, 
    wdc
):
    from gurobipy import Model, GRB, quicksum
    import pandas as pd

    # ========================
    #  1. 创建模型
    # ========================
    model = Model("Power_Management")

    # ========================
    #  2. 定义变量索引和集
    # ========================
    I = range(len(instance)) 
    F = {i: range(len(instance[i]['operations'])) for i in I}  # 每个进程的操作数量
    operations_info = {i: instance[i]['operations'] for i in I}
    operation_power = {
        i: {k: operations_info[i][k]['power'] for k in F[i]} 
        for i in I
    }

    # ========================
    #  3. 定义决策变量
    # ========================
    # 3.1 太阳能相关
    G_solar = model.addVars(range(start_hour, end_hour), lb=0, name="G_solar") # generate power
    P_solar = model.addVars(range(start_hour, end_hour), lb=0, name="P_solar") # power for data center
    R_solar = model.addVars(range(start_hour, end_hour), lb=0, name="R_solar") # power for battery
    W_solar = model.addVars(range(start_hour, end_hour), lb=0, name="W_solar") # power for selling back to the grid

    for t in range(start_hour, end_hour):
        G_solar[t].ub = G_max_solar[t]
        P_solar[t].ub = G_max_solar[t]
        R_solar[t].ub = G_max_solar[t]
        W_solar[t].ub = G_max_solar[t]

    # 3.2 电网相关
    G_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="G_grid") # generate power
    P_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="P_grid") # power for data center
    R_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="R_grid") # power for battery
    W_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="W_grid") # power for selling back to the grid

    # 3.3 电池相关
    D_dc = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_dc")
    D_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_grid")
    ESD = model.addVars(range(start_hour, end_hour+1), lb=0, ub=E_max, name="ESD")

    # 3.4 GPU 功率相关
    # y[t,i,k]：表示在时间槽 t，进程 i 的操作 k 是否执行
    y = model.addVars(range(start_hour, end_hour), I, F[0], vtype=GRB.BINARY, name="y")
    gpu_power = model.addVars(range(start_hour, end_hour), lb=0, ub=max(GPU_power_options), name="gpu_power")
    power_selection = model.addVars(range(start_hour, end_hour), len(GPU_power_options), vtype=GRB.BINARY, name="power_selection")

    # 3.5 每个时间槽完成量
    completion_per_slot = model.addVars(range(start_hour, end_hour), name="completion_per_slot", vtype=GRB.CONTINUOUS)

    # ========================
    #  4. 目标函数
    # ========================
    # 最小化总用电成本：电网买电 - 卖回电网的收益
    objective = quicksum(
        (G_grid[t]/1000 * electricity_price[t] - W_grid[t]/1000 * sell_back_price) 
        for t in range(start_hour, end_hour)
    )
    model.setObjective(objective, sense=GRB.MINIMIZE)

    # ========================
    #  5. 约束条件
    # ========================
    # 5.1 太阳能供电平衡: 太阳能发电 = 数据中心 + 电池充电 + 卖回电网
    model.addConstrs(
        (P_solar[t] + R_solar[t] + W_solar[t] == G_solar[t])
        for t in range(start_hour, end_hour)
    )

    # 5.2 电网供电平衡: 电网总输出 = 数据中心 + 电池充电
    model.addConstrs(
        (P_grid[t] + R_grid[t] == G_grid[t])
        for t in range(start_hour, end_hour)
    )

    # 5.3 卖回电网功率平衡
    model.addConstrs(
        (W_solar[t] + beta * D_grid[t] == W_grid[t])
        for t in range(start_hour, end_hour)
    )

    # 5.4 电池容量限制
    model.addConstrs(
        (R_solar[t] + R_grid[t] <= (E_max - ESD[t]))
        for t in range(start_hour, end_hour)
    )

    # 5.5 电池容量更新
    model.addConstrs(
        (ESD[t] == (1 - theta) * (ESD[t-1] - D_dc[t-1] - D_grid[t-1] + alpha * (R_solar[t-1] + R_grid[t-1])))
        for t in range(start_hour+1, end_hour+1)
    )

    # 5.6 放电功率不能超过当前电池容量
    model.addConstrs(
        (D_dc[t] + D_grid[t] <= ESD[t]) 
        for t in range(start_hour, end_hour)
    )
    model.addConstrs(
        (D_dc[t] + D_grid[t] >= 0)
        for t in range(start_hour, end_hour)
    )

    # 5.7 每个时间槽只执行一个 operation
    for t in range(start_hour, end_hour):
        model.addConstr(
            completion_per_slot[t] == quicksum(y[t, i, k] * contributions[i] for i in I for k in F[i]),
            name=f"CompletionTime_{t}"
        )
        model.addConstr(
            quicksum(y[t, i, k] for i in I for k in F[i]) == 1,
            name=f"OneOperationAtTime_{t}"
        )

    # 5.8 总的完成度至少达到 1
    model.addConstr(
        quicksum(completion_per_slot[t] for t in range(start_hour, end_hour)) >= 1,
        name="CompleteJob"
    )

    # 5.9 数据中心用电平衡与 GPU 功率约束
    model.addConstrs(
        (
            P_solar[t] + P_grid[t] + beta * D_dc[t] 
            >= quicksum(y[t, i, k] * instance[i]['operations'][k]['power'] for i in I for k in F[i])
        )
        for t in range(start_hour, end_hour)
    )
    model.addConstrs(
        (P_solar[t] + P_grid[t] + beta * D_dc[t] >= gpu_power[t])
        for t in range(start_hour, end_hour)
    )
    model.addConstrs(
        (
            quicksum(power_selection[t, j] * GPU_power_options[j] for j in range(len(GPU_power_options))) 
            == gpu_power[t]
        )
        for t in range(start_hour, end_hour)
    )
    model.addConstrs(
        (
            quicksum(power_selection[t, j] for j in range(len(GPU_power_options))) == 1
        )
        for t in range(start_hour, end_hour)
    )
    model.addConstrs(
        (
            quicksum(y[t, i, k] * operation_power[i][k] for i in I for k in F[i]) <= gpu_power[t]
        )
        for t in range(start_hour, end_hour)
    )

    # 5.10 初始条件: 电池初始能量为 0
    ESD[start_hour].setAttr(GRB.Attr.LB, 0)
    ESD[start_hour].setAttr(GRB.Attr.UB, 0)

    # ========================
    #  6. 定义回调函数  # <-- 新增或修改
    # ========================
    def my_callback(model, where):
        """
        在求解过程中，一旦发现某一个可行解满足 total_completion >= 1 就立即终止。
        如果你希望得到最优解，则请不要使用该回调来强制终止。
        """
        if where == GRB.Callback.MIPSOL:
            # 获取当前可行解中 completion_per_slot 的值
            completion_sum = 0.0
            for t in range(start_hour, end_hour):
                val = model.cbGetSolution(model.getVarByName(f"completion_per_slot[{t}]"))
                completion_sum += val

            if completion_sum >= 1.0:
                # 一旦发现完成度 >= 1, 就终止求解
                model.terminate()  # <-- 关键的中断操作

    # ========================
    #  7. 求解模型时挂载回调  # <-- 新增或修改
    # ========================
    # 如果你想要找到可行解就立即退出，则使用回调
    # 如果仍想要最优解，则可以直接 model.optimize() 而不传回调
    model.optimize(my_callback)  # <-- 关键点：加了回调

    # ========================
    #  8. 获取并输出结果
    # ========================
    # （如果求解在满足 total_completion >= 1 后提早终止，
    #  则只会得到一个可行解，不一定是最优解）
    total_completion = sum(completion_per_slot[t].X for t in range(start_hour, end_hour))
    minimum_cost = model.objVal
    results = {}

    if model.status == GRB.Status.OPTIMAL or model.status == GRB.Status.USER_OBJ_LIMIT or model.status == GRB.Status.TIME_LIMIT or model.status == GRB.Status.INTERRUPTED:
        # 对于使用回调强行终止，也可能出现 status=INTERRUPTED 等
        results = {
            'status': 'Solution Found',
            'details': [],
            'total_job_completion': f"{total_completion:.2%}",  # Format as a percentage
            'minimum_cost': minimum_cost
        }
        for t in range(start_hour, end_hour):
            time_details = {
                'time': t,
                'G_grid': G_grid[t].X,
                'W_grid': W_grid[t].X,
                'P_grid': P_grid[t].X,
                'R_grid': R_grid[t].X,
                'G_solar': G_solar[t].X,
                'W_solar': W_solar[t].X,
                'P_solar': P_solar[t].X,
                'R_solar': R_solar[t].X,
                'D_dc': D_dc[t].X,
                'D_grid': D_grid[t].X,
                'GPU_power': gpu_power[t].X,
                'operations': [],
                'job_completion': completion_per_slot[t].X 
            }
            for i in I:
                for k in range(len(operation_power[i])):
                    if y[t, i, k].X > 0.5:
                        time_details['operations'].append(f"Process {i} is executed")
            results['details'].append(time_details)
    else:
        results['status'] = "No Optimal Solution Found"

    energy_dispatch = results['details']
    energy_dispatch_df = pd.DataFrame(energy_dispatch)

    energy_dispatch_df['total_job_completion'] = f"{total_completion:.2%}"
    energy_dispatch_df['minimum_cost'] = f"${minimum_cost:.2f}"

    # save the results to result_dir
    # energy_dispatch_df.to_csv(os.path.join(result_dir, 'Duration{}_start_hour{}_wdc{}.csv'.format(end_hour-start_hour, start_hour, wdc)), index=False)
    energy_dispatch_df.to_csv(os.path.join(fix_panel_dir, 'Duration{}_start_hour{}.csv'.format(end_hour-start_hour, start_hour)), index=False)
    # energy_dispatch_df.to_csv('Duration{}_start_hour{}_wdc{}.csv'.format(end_hour-start_hour, start_hour, wdc), index=False)
    
    return energy_dispatch_df

In [17]:
# def solve_gurobi(start_hour, end_hour, instance,contributions,E_max, alpha, beta, theta, G_max_solar, G_max_grid, GPU_power_options, electricity_price, sell_back_price, wdc):
    
#     model = Model("Power_Management")

#     # 创建变量
#     # Hours = end_hour - start_hour  # 确保Hours是定义的持续时间
#     I = range(len(instance)) # 进程的数量
#     F = {i: range(len(instance[i]['operations'])) for i in I} # 每个进程的操作数量
#     operations_info = {i: instance[i]['operations'] for i in I}
#     operation_power = {i: {k: operations_info[i][k]['power'] for k in F[i]} for i in I}
#     # operation_time = {i: {k: operations_info[i][k]['duration'] for k in F[i]} for i in I}
#     # process_max_time = {i: instance[i]['max_time'] for i in I}

#     G_solar = model.addVars(range(start_hour, end_hour), lb=0, name="G_solar")
#     P_solar = model.addVars(range(start_hour, end_hour), lb=0, name="P_solar")
#     R_solar = model.addVars(range(start_hour, end_hour), lb=0, name="R_solar")
#     W_solar = model.addVars(range(start_hour, end_hour), lb=0, name="W_solar")

#     for t in range(start_hour, end_hour):
#         G_solar[t].ub = G_max_solar[t]
#         P_solar[t].ub = G_max_solar[t]
#         R_solar[t].ub = G_max_solar[t]
#         W_solar[t].ub = G_max_solar[t]

#     # 电网
#     G_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="G_grid")
#     P_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="P_grid")
#     R_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="R_grid")
#     W_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=G_max_grid, name="W_grid")

#     # 电池
#     D_dc = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_dc")
#     D_grid = model.addVars(range(start_hour, end_hour), lb=0, ub=E_max, name="D_grid")
#     ESD = model.addVars(range(start_hour, end_hour+1), lb=0, ub=E_max, name="ESD")

#     # GPU功率
#     # y = model.addVars(range(start_hour, end_hour), I, {i: F[i] for i in I}, vtype=GRB.BINARY, name="y") # 操作选择变量 第t时刻第i个进程的第k个操作是否执行
#     y = model.addVars(range(start_hour, end_hour), I, F[0], vtype=GRB.BINARY, name="y")
#     gpu_power = model.addVars(range(start_hour, end_hour), lb=0, ub=max(GPU_power_options), name="gpu_power")
#     power_selection = model.addVars(range(start_hour, end_hour), len(GPU_power_options), vtype=GRB.BINARY, name="power_selection")

#     # 目标函数：最小化总电力成本
#     objective =  quicksum((G_grid[t] * electricity_price[t] - W_grid[t] * sell_back_price) for t in range(start_hour, end_hour))
#     model.setObjective(objective, sense=GRB.MINIMIZE)

#     model.addConstrs(P_solar[t] + R_solar[t] + W_solar[t] == G_solar[t] for t in range(start_hour, end_hour)) # 太阳能给出去的总功率 = data center + battery + back to grid 
#     model.addConstrs(P_grid[t] + R_grid[t] == G_grid[t] for t in range(start_hour, end_hour)) # 电网给出去的总功率 = data center + battery
#     model.addConstrs(W_solar[t] + beta * D_grid[t] == W_grid[t] for t in range(start_hour, end_hour)) # 卖回电网的功率等于太阳能卖回电网的功率加上电池卖回电网的功率
#     model.addConstrs(R_solar[t]+R_grid[t] <= (E_max - ESD[t]) for t in range(start_hour, end_hour)) # 电池容量限制
#     model.addConstrs(ESD[t] == (1-theta) * (ESD[t-1] - D_dc[t-1] - D_grid[t-1] + alpha * (R_solar[t-1] + R_grid[t-1])) for t in range(start_hour+1, end_hour+1)) # 电池容量更新
#     model.addConstrs(D_dc[t] + D_grid[t] <= ESD[t] for t in range(start_hour, end_hour)) # 电池放电功率不得超过电池容量
#     model.addConstrs(D_dc[t] + D_grid[t] >= 0 for t in range(start_hour, end_hour)) # 电池放电功率不得为负

#     # 每个时间槽的job完成量变量
#     completion_per_slot = model.addVars(range(start_hour, end_hour), name="completion_per_slot", vtype=GRB.CONTINUOUS)

#     # 每个时间槽中只能执行一个operation
#     for t in range(start_hour, end_hour):
#         model.addConstr(completion_per_slot[t] == quicksum(y[t, i, k] * contributions[i] for i in I for k in F[i]), name=f"CompletionTime_{t}")
#         model.addConstr(quicksum(y[t, i, k] for i in I for k in F[i]) == 1, name=f"OneOperationAtTime_{t}")

#     # 定义完成率变量
#     model.addConstr(quicksum(completion_per_slot[t] for t in range(start_hour, end_hour)) >= 1, name="CompleteJob")

#     model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= quicksum(y[t, i, k] * instance[i]['operations'][k]['power'] for i in I for k in F[i]) for t in range(start_hour, end_hour)), "PowerBalance")
#     model.addConstrs((P_solar[t] + P_grid[t] + beta * D_dc[t] >= gpu_power[t] for t in range(start_hour, end_hour)), "DataCenterPowerSupplyMatchesGPUPower")
#     model.addConstrs((quicksum(power_selection[t, j] * GPU_power_options[j] for j in range(len(GPU_power_options))) == gpu_power[t] for t in range(start_hour, end_hour)),"SelectGPUPower")
#     model.addConstrs((quicksum(power_selection[t, j] for j in range(len(GPU_power_options))) == 1 for t in range(start_hour, end_hour)),"OnePowerOption")

#     model.addConstrs(
#         (quicksum(y[t, i, k] * operation_power[i][k] for i in I for k in F[i]) <= gpu_power[t] 
#         for t in range(start_hour, end_hour)),"GPUPowerLimit")

#     # 初始条件和容量限制
#     ESD[start_hour].setAttr(GRB.Attr.LB, 0)
#     ESD[start_hour].setAttr(GRB.Attr.UB, 0)

#     # 求解模型
#     model.optimize()

#     # 输出结果
#     total_completion = sum(completion_per_slot[t].X for t in range(start_hour, end_hour))
#     minimum_cost = model.objVal
#     results = {}
#     if model.status == GRB.Status.OPTIMAL:
#         # results['status'] = "Optimal Solution Found"
#         # results['details'] = []
#         results = {
#         'status': 'Optimal Solution Found',
#         'details': [],
#         'total_job_completion': f"{total_completion:.2%}",  # Format as a percentage
#         'minimum_cost': minimum_cost
#     }
#         for t in range(start_hour, end_hour):
#             time_details = {
#                 'time': t,
#                 'G_grid': G_grid[t].X,
#                 'W_grid': W_grid[t].X,
#                 'P_grid': P_grid[t].X,
#                 'R_grid': R_grid[t].X,
#                 'G_solar': G_solar[t].X,
#                 'W_solar': W_solar[t].X,
#                 'P_solar': P_solar[t].X,
#                 'R_solar': R_solar[t].X,
#                 'D_dc': D_dc[t].X,
#                 'D_grid': D_grid[t].X,
#                 'GPU_power': gpu_power[t].X,
#                 'operations': [],
#                 'job_completion': completion_per_slot[t].X 
        
#             }
#             for i in I:
#                 for k in range(len(operation_power[i])):
#                     if y[t, i, k].X > 0.5:
#                         time_details['operations'].append(f"Process {i} is executed")
#             results['details'].append(time_details)
#     else:
#         results['status'] = "No Optimal Solution Found"


#     energy_dispatch = results['details']
#     energy_dispatch_df = pd.DataFrame(energy_dispatch)

#     energy_dispatch_df['total_job_completion'] = f"{total_completion:.2%}"
#     energy_dispatch_df['minimum_cost'] = f"${minimum_cost:.2f}"

#     energy_dispatch_df.to_csv('Duration{}_start_hour{}_wdc{}.csv'.format(end_hour-start_hour, start_hour, wdc), index=False)
    
#     return energy_dispatch_df



In [18]:
# Create a list to store the start hour of each month
# For a year with 8760 hours (365 days * 24 hours)
# Calculate the start hour for each month based on number of days in each month
month_starts = [0]  # January starts at hour 0
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

for days in days_in_month[:-1]:  # Exclude December since we don't need its end
    next_month = month_starts[-1] + (days * 24)
    month_starts.append(next_month)

print("Start hour of each month:")
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, hour in zip(months, month_starts):
    print(f"{month}: {hour}")

print(f'The month starts are: {month_starts}')

Start hour of each month:
Jan: 0
Feb: 744
Mar: 1416
Apr: 2160
May: 2880
Jun: 3624
Jul: 4344
Aug: 5088
Sep: 5832
Oct: 6552
Nov: 7296
Dec: 8016
The month starts are: [0, 744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016]


In [19]:
process_operations = generate_instance(power_level=power_level, time_requirement=time_requirement)

In [20]:
# contributions = [1/32, 1/28, 1/26, 1/25, 1/24, 1/23, 1/22]
contributions = [np.round(1/i,3) for i in time_requirement]
# print(contributions)

# start_hour = 0
# Hours = 20
# end_hour = start_hour + Hours
# print(cur_dir)
# read the csv file from the data_dir
# read the files in the data_dir
files = os.listdir(data_dir)
# remove the file that the name did not contain wdc
files = sorted([file for file in files if 'wdc' in file])
files1000 = files[-1]
print(files1000)

electricity_price = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.35, 0.45, 0.45, 0.45, 0.45,
                     0.45, 0.45, 0.45, 0.45, 0.45, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.25]*365

sell_back_price = 0.05

# print(files)


for start_hour in month_starts:
    for time in range(min_time, max_time+1):
        end_hour = start_hour + time
        data_dir = os.path.join(parent_dir, 'PVData')
        data_dir = os.path.join(data_dir, files1000)
        print(f'The path is: {data_dir}')
        solar_data = pd.read_csv(data_dir)
        G_max_solar = solar_data['AC System Output (W)'][1:169].tolist()
        result = solve_gurobi(
            start_hour = start_hour,
            end_hour = end_hour,
            instance = process_operations,
            contributions = contributions,
            E_max=250,
            alpha=0.9,
            beta=0.9,
            theta=0.005,
            G_max_solar=G_max_solar,
            G_max_grid= 1000,
            GPU_power_options=power_level,
            electricity_price= electricity_price,
            sell_back_price=sell_back_price,
            wdc = 1000
        )
        print(result)
    # GPU_power_options = [100, 125, 150, 175, 200, 225, 250]
    # electricity_price = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.35, 0.45, 0.45, 0.45, 0.45,
    #                      0.45, 0.45, 0.45, 0.45, 0.45, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.25]*7
    # sell_back_price = 0.05

pvwatts_7sqm_1000wdc.csv
The path is: /Users/dtjgp/Projects/GreenAI/SolarPanelData/PVData/pvwatts_7sqm_1000wdc.csv
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D5055b)

CPU model: Apple M3 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 197 rows, 9521 columns and 21686 nonzeros
Model fingerprint: 0x4b5d873b
Variable types: 183 continuous, 9338 integer (9338 binary)
Coefficient statistics:
  Matrix range     [4e-02, 3e+02]
  Objective range  [5e-05, 5e-04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 2e+02]
Presolve removed 55 rows and 9011 columns
Presolve time: 0.01s
Presolved: 142 rows, 510 columns, 1402 nonzeros
Variable types: 76 continuous, 434 integer (420 binary)

Root relaxation: objective 8.285375e-01, 162 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

IndexError: list index out of range

In [ ]:
grid_only_power = max(power_level)  # maximum GPU power needed
print(f'The maximum GPU power needed is: {grid_only_power}W')
total_hours = 13.9

# Calculate the total cost when only using grid power
grid_only_cost = 0
for t in range(math.ceil(total_hours)):
    grid_only_cost += (grid_only_power/1000) * electricity_price[t+8]

print(f"If only using grid power at {grid_only_power}W for {total_hours} hours:")
print(f"Total cost: ${grid_only_cost:.2f}")

# calculate the total consumption of the traing process
total_consumption = grid_only_power * math.ceil(total_hours)
print(f'The total consumption of the training process is: {total_consumption}Wh')

The maximum GPU power needed is: 320W


NameError: name 'electricity_price' is not defined

In [11]:
csv_files = os.listdir(result_dir)
# sort the csv files
csv_files = sorted(csv_files)

# create a dataframe to save the needed results, first column is the configuration of the training process, second conlumn is the total grid consumption, third column is the total cost
results = pd.DataFrame(columns=['Configuration', 'Total Grid Consumption', 'Total Cost'])
for i in range(len(csv_files)):
    csv_file = csv_files[i]           
    start_hour = csv_file.split('_')[2][4:]  
    # print(start_hour)
    duration = csv_file.split('_')[0][-2:]
    wdc = csv_file.split('_')[-1][3:]
    name = f's{start_hour}_d{duration}_wdc{wdc}'
    print(name)
    # read the csv file
    data = pd.read_csv(os.path.join(result_dir, csv_file))
    # get the total grid consumption 
    total_grid_consumption = data['G_grid'].sum()
    print(f'The total grid consumption is: {np.round(total_grid_consumption,3)}Wh')
    # get the total cost
    total_cost = data['minimum_cost'][0]
    print(f'The total cost is: {total_cost}')
    # add the results to the dataframe
    results.loc[len(results)] = [name, total_grid_consumption, total_cost]
    

# save the results to the current directory
results.to_csv('results.csv', index=False)

s0_d14_wdc100.csv
The total grid consumption is: 2816.991Wh
The total cost is: $0.90
s0_d14_wdc1000.csv
The total grid consumption is: 2651.439Wh
The total cost is: $0.83
s0_d14_wdc200.csv
The total grid consumption is: 2797.372Wh
The total cost is: $0.89
s0_d14_wdc300.csv
The total grid consumption is: 2777.751Wh
The total cost is: $0.89
s0_d14_wdc400.csv
The total grid consumption is: 2758.132Wh
The total cost is: $0.88
s0_d14_wdc500.csv
The total grid consumption is: 2738.513Wh
The total cost is: $0.87
s0_d14_wdc600.csv
The total grid consumption is: 2718.893Wh
The total cost is: $0.86
s0_d14_wdc700.csv
The total grid consumption is: 2699.364Wh
The total cost is: $0.85
s0_d14_wdc800.csv
The total grid consumption is: 2679.757Wh
The total cost is: $0.84
s0_d14_wdc900.csv
The total grid consumption is: 2660.149Wh
The total cost is: $0.83
s12_d14_wdc100.csv
The total grid consumption is: 4471.128Wh
The total cost is: $1.63
s12_d14_wdc1000.csv
The total grid consumption is: 4399.023Wh
T

In [13]:
# get the minimum cost configuration from the results
min_cost = results['Total Cost'].min()
min_cost_config = results[results['Total Cost'] == min_cost]
print(f'The minimum cost configuration is: {min_cost_config}')

The minimum cost configuration is:            Configuration  Total Grid Consumption Total Cost
331  s20_d19_wdc1000.csv             2304.376149      $0.70


In [17]:
# calculate the percentage of the minimum cost compared to the grid only cost
min_cost = min_cost_config['Total Cost'].values[0]
print(f'The minimum cost is: {min_cost}')
# Extract the numeric value from min_cost string by removing '$'
min_cost_value = float(min_cost.replace('$', ''))
cost_reduction = (grid_only_cost - min_cost_value) / grid_only_cost * 100
print(f'The cost reduction is: {cost_reduction:.2f}%')

# calculate the energy reduction
min_consumption = min_cost_config['Total Grid Consumption'].values[0]
print(f'The minimum grid consumption is: {min_consumption}')
energy_reduction = (total_consumption - min_consumption) / total_consumption * 100
print(f'The energy reduction is: {energy_reduction:.2f}%')

The minimum cost is: $0.70
The cost reduction is: 62.28%
The minimum grid consumption is: 2304.376148705023
The energy reduction is: 48.56%


In [12]:
# result = solve_gurobi(
#     start_hour = start_hour,
#     end_hour = end_hour,
#     instance = process_operations,
#     contributions = contributions,
#     E_max=250,
#     alpha=0.9,
#     beta=0.9,
#     theta=0.005,
#     G_max_solar=G_max_solar,
#     G_max_grid= 1000,
#     GPU_power_options=power_level,
#     electricity_price= electricity_price,
#     sell_back_price=0.05
#     wdc
# )
